In [17]:
import numpy as np
from functools import reduce
from random import random, seed
from queue import PriorityQueue
from collections import namedtuple
from scipy.stats import binom
import scipy.special as sp

In [18]:
seed(1)
PROBABILITY = 0.3
PROBLEM_SIZE = 75
NUM_SETS = 75

# 20% prob of being true using array compriansion and then converting to nparray
SETS = tuple(
    np.array([random() < PROBABILITY for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)

## removes double sets
SETS = set(tuple(tuple(set_) for set_ in SETS))
SETS = list(np.array(set_) for set_ in SETS)

## sort elements from the one with most zeros
SETS.sort(key=lambda x: -sum(x))
SETS = tuple(SETS)
NUM_SETS = len(SETS)

# create named tuple
State = namedtuple("State", ["taken", "not_taken"])

In [19]:
def goal_check(state):
    if state[0] == list() or state[0] == set():
        return False
    return np.all(reduce(np.logical_or, [SETS[i] for i in state[0]]))


assert goal_check((list(range(NUM_SETS)), list())), "Problem not solvable"


### DETERMINISTIC HEURISTIC ###
def h(state):
    """
    Deterministic heusìristic function
    """
    reduction = reduce(np.logical_or, [SETS[i] for i in state[0]])
    n_zero = PROBLEM_SIZE - np.sum(reduction)
    if n_zero < 2:
        return n_zero
    else:
        pos_zeros = [i for i, item in enumerate(reduction) if item == False]
        a = 0
        for tile in state.not_taken:
            if reduce(np.logical_or, SETS[tile][pos_zeros]) == True:
                a += 1
        if a == 0:
            return 2 + 1 - n_zero / PROBLEM_SIZE
        else:
            return 1
def dist(state):
    return len(state.taken)

### A STAR DISTANCE MESURE ###
def a_star_distance(state):
    return len(state.taken) + h(state)

In [20]:
### PROBABILISTIC HEURISTIC ###


def prob_estimation(n, *, flag=1):
    if flag == 1:
        return PROBABILITY**n
    elif flag == 2:
        p = 0
        for i in range(1, n):
            p += binom.pmf(i, n, PROBABILITY) * PROBABILITY ** (n - i)
        return p

global_LookupTable_p = dict()

def h_p(state):
    """
    Stochastic heuristic function based on the binomial distribution
    alpha: probability threshold
    """
    alpha = 0.05
    reduction = reduce(np.logical_or, [SETS[i] for i in state[0]])
    n_zero = PROBLEM_SIZE - np.sum(reduction)
    if n_zero < 2:
        return n_zero
    else:
        n_not_taken = len(state.not_taken)
        if (1 - (1 - prob_estimation(n_zero, flag=1)) ** n_not_taken) > alpha:

            return 1 + 1 - n_zero / PROBLEM_SIZE
        
        elif (1 - (1 - prob_estimation(n_zero, flag=2)) ** sp.binom(n_not_taken, 2)) > alpha:

            return 2 + 1 - n_zero / PROBLEM_SIZE
        else:
            return 3 + 1 - n_zero / PROBLEM_SIZE
        
def h_p2(state):
    """
    Stochastic heuristic function based on the binomial distribution
    alpha: probability threshold
    """
    alpha = 0.05
    reduction = reduce(np.logical_or, [SETS[i] for i in state[0]])
    n_zero = PROBLEM_SIZE - np.sum(reduction)
    if n_zero < 2:
        return n_zero
    else:
        n_not_taken = len(state.not_taken)

        if (n_zero, 1) not in global_LookupTable_p:
            global_LookupTable_p[(n_zero, 1)] = prob_estimation(n_zero, flag=1)
        if (n_zero, 2) not in global_LookupTable_p:
            global_LookupTable_p[(n_zero, 2)] = prob_estimation(n_zero, flag=2)

        if (1 - (1 - global_LookupTable_p[(n_zero, 1)]) ** n_not_taken) > alpha:

            return 1 - n_zero / PROBLEM_SIZE
        
        elif (1 - (1 - global_LookupTable_p[(n_zero, 2)]) ** sp.binom(n_not_taken, 2)) > alpha:

            return 1 + 1 - n_zero / PROBLEM_SIZE
        else:
            return 2 + 1 - n_zero / PROBLEM_SIZE

def a_star_distance_p(state):
    return len(state.taken) + h_p(state)

### fusion test ###
def a_star_mix(state):
    if len(state.not_taken) > 100:
        return len(state.taken) + h_p(state)
    else:
        return len(state.taken) + h(state)

In [21]:
def action_list_func(state):
    return [action_ for action_ in state.not_taken]


def action_func(state, current_action):
    new_taken = state.taken.copy()
    new_taken.append(current_action)
    return State(new_taken, state[1].copy()[current_action + 1 :])


def A_star(init_state, action_list_func, action_func, goal_check, valid_dist):
    frontier = PriorityQueue()
    counter = 0
    current_state = init_state
    while not goal_check(current_state):
        counter += 1
        action_list = action_list_func(current_state)
        for action in action_list:
            new_state = action_func(current_state, action)
            # priority = distance(new_state) + heuristic(new_state)
            priority = valid_dist(new_state)
            frontier.put((priority, new_state))
        _, current_state = frontier.get()
    
    return counter, current_state

In [22]:
# probilistic h
counter, sol_state = A_star(State(list(), list(range(NUM_SETS))), action_list_func, action_func, goal_check, a_star_distance_p)
print(f'Solved in {counter:,} steps, with {len(sol_state.taken)} sets')
print(sol_state.taken)
sum(sum([SETS[i] for i in sol_state.taken]))

Solved in 3,004 steps, with 5 sets
[0, 3, 5, 11, 46]


141

In [23]:
# mixed strategies
counter, sol_state = A_star(State(list(), list(range(NUM_SETS))), action_list_func, action_func, goal_check, a_star_mix)
print(f'Solved in {counter:,} steps, with {len(sol_state.taken)} sets')
print(sol_state.taken)
sum(sum([SETS[i] for i in sol_state.taken]))


Solved in 12,030 steps, with 5 sets
[0, 3, 5, 11, 46]


141

In [24]:
# deterministic h
counter, sol_state = A_star(State(list(), list(range(NUM_SETS))), action_list_func, action_func, goal_check, a_star_distance)
print(f'Solved in {counter:,} steps, with {len(sol_state.taken)} sets')
print(sol_state.taken)
sum(sum([SETS[i] for i in sol_state.taken]))


Solved in 12,030 steps, with 5 sets
[0, 3, 5, 11, 46]


141